In [1]:
import pyodbc
import pandas as pd
import numpy as np 
import re

import urllib
from sqlalchemy import create_engine

from tqdm import tqdm
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:95% !important; }</style>"))

# pd.set_option('display.max_columns', None)
from termdata import terms_by_type, terms_by_country #this termdata.py copy from lib cleanco
import functools
import operator

import datetime


In [2]:
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
# pd.set_option('display.width', 1000)


In [2]:
params_test3 = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                       "SERVER=name;"
                                       "DATABASE=name;"
                                       "UID=name;"
                                       "PWD=name")

params_DK = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                    "SERVER=name;"
                                    "DATABASE=name;"
                                    "UID=name;"
                                    "PWD=name")

params_VM = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                    "SERVER=name;"
                                    "DATABASE=name;"
                                    "UID=name;"
                                    "PWD=name")

params_vendor_orgill = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                                "SERVER=name;"
                                                "DATABASE=name;"
                                                "UID=name;"
                                                "PWD=name")



In [3]:
ts = functools.reduce(operator.iconcat, terms_by_type.values(), [])
cs = functools.reduce(operator.iconcat, terms_by_country.values(), [])
stop_words = set(ts + cs)
def remove_stop_words(string):
    """Remove stop words from string """
    # ts = functools.reduce(operator.iconcat, terms_by_type.values(), [])
    # cs = functools.reduce(operator.iconcat, terms_by_country.values(), [])
    # stop_words = set(ts + cs)
    return ' '.join(filter(lambda x: x not in stop_words, string.split()))
    # if string:
    #     return ' '.join(filter(lambda x: x not in stop_words, string.split()))
    # else:
    #     return pd.NA

def normalize_name(df, name_columns=['raw_manuf_v', 'manuf_m'], suffix = '_cl'):
    """The column 'clean' is created with normalized names, for example,
        everything except letters and numbers are deleted ; business stop words are deleted"""

    # df = df.copy()
    for name_col in name_columns:
        name_column_clean = name_col + suffix
        df[name_column_clean] = df.loc[:,name_col].str.lower()
        df[name_column_clean].replace(to_replace ='[\W]+', value = ' ', regex = True, inplace=True)
        df.loc[df[name_column_clean].notna(), name_column_clean] = df.loc[df[name_column_clean].notna(), name_column_clean].apply(remove_stop_words)
        df[name_column_clean].replace(to_replace ='[\W]+', value = ' ', regex = True, inplace=True)
        df.loc[df[name_column_clean].notna(), name_column_clean] = df.loc[df[name_column_clean].notna(), name_column_clean].apply(remove_stop_words) #because after deleting '[\W]+' maybe will find pattern remove_stop_words, need two times remove_stop_words
        df[name_column_clean].replace(to_replace ='[\W]+', value = ' ', regex = True, inplace=True)
        df.loc[df[name_column_clean] == ' ', name_column_clean] = pd.NA

    # column_patern = name_column1 + '_pat'
    # # df[column_patern] = df[name_column1_clean].str.replace(' ', '.+', regex=False)
    # df[column_patern] = df[name_column1_clean].apply(lambda x: '.+'.join(['('+word+')' for word in x.split()]))

    # df['clean'].replace(to_replace =' ', value = '', regex = True, inplace=True)
    # df['clean'] = df['clean'].str.strip()

    return df

def normalize_upc(df, col_upc, new_col_name):
    df[new_col_name] = df[col_upc].str.strip()
    df[new_col_name] = df[new_col_name].str.lower()
    # df['upc_mod'] = df['upc_mod'].str.removeprefix('s')
    # df['upc_mod'] = df['upc_mod'].str.removeprefix('0')
    df[new_col_name].replace(to_replace ='^s{0,20}', value = '', regex = True, inplace=True)
    df[new_col_name].replace(to_replace ='^0{0,20}', value = '', regex = True, inplace=True)
    df[df[new_col_name] == ''] = pd.NA

    return df

def replace_pattern(string, modify_degree):
    # old_all_pattern = {'inch':'[\d\.]inchx?', 'inc':'[\d\.]incx?', 'in':'[\d\.](in)x?', 'us':'us\d+', '0':'[\.\,][1-9]*0+', 'x':'[\d\.](in(ch*)*)*x\d'}
    all_patterns = {'in(c(he?s?)?)?':'[^a-z]in(c(he?s?)?)?[^a-z0-9]', 'us':'[^a-z]us\d+', '0':'[\.\,][1-9]*0+$', 'x':'[\d\. ]x\d'}
    patterns_1 = {'0':'[\.\,][1-9]*0+$'}
    # need drop 0 in begin 
    #problem '33JXUS32D'
    #problem сопоставить нат.дроби или десятичная
    
    if modify_degree == 'low':
        patterns = patterns_1
    else:
        patterns = all_patterns
    
    
    string = str(string).lower()
    
    for key, pattern in patterns.items():
        match = re.search(pattern, string)
        if match is None:
            continue
        else:
            string = re.sub(pattern, lambda match_obj:re.sub(key, '', match_obj[0]) , string)
 
    return re.sub('[\W]+', '', string)

#example
string = '299 US26D'
# string ='55.0001'
replace_pattern(string, 'high')

'29926d'

In [4]:
def filter_short_numeric_mpn(df, name_col='mpn_mod', lenght=4, lenght_num=6):
    """filter string are only numerics with lenght has no more lenght_num """
    df_lenght = df.loc[df[name_col].str.len() >= lenght].copy()

    alpha_numeric = df_lenght.loc[~df_lenght[name_col].str.isnumeric()]
    numeric = df_lenght.loc[df_lenght[name_col].str.isnumeric()]
    numeric_len = numeric.loc[numeric[name_col].str.len() >= lenght_num]
        
    return pd.concat([alpha_numeric, numeric_len])

def fix_short_upc(df, name_col='UPC', lenght=12, min_lenght=9, debug=False):
    """add '0' to the begin string untill lenght = 12 """
    df = df.copy()
    flt_non_lenght_idx = df.loc[(df[name_col].str.len() != lenght) & df['UPC'].notna()].index

    for n in range(min_lenght, lenght):
        n_idx = df.loc[df[name_col].str.len() == n].index
        df.loc[n_idx, name_col] = '0'*(lenght - n) + df.loc[n_idx, name_col]

    if not df.loc[(df[name_col].str.len() != lenght) & df['UPC'].notna()].empty:
        print('Alert! Exist uncorrected upc')

    if debug:
        return df.loc[flt_non_lenght_idx]
    else:
        return df

def get_first_non_null(dfrow, columns_to_search):
    """def like sql coalise"""
    for c in columns_to_search:
        if pd.notnull(dfrow[c]) and dfrow[c] !=0:
            return dfrow[c]
    # for execute def
#     cols_to_search = ['ret_price_m', 'price_m']
#     df3['price_m_mod'] = df3.apply(lambda x: get_first_non_null(x, cols_to_search), axis=1)
    return None

upc_v_mod = 'upc_v_mod'
upc_m_mod = 'upc_m_mod'
mpn_v_mod = 'mpn_v_mod'
mpn_m_mod = 'mpn_v_mod'

def find_conflicts(slave, master, ratio_prise=0.2, ratio_prise_manuf=0.5):
    # cols = ['prod_id_v', 'prod_id_m', 'mpn_v', 'mpn_m', 'upc_v', 'upc_m', 'manuf_v', 'manuf_m', 'price_v',
    #         'ret_price_m', 'price_m', 'title_v', 'title_m', 'manuf_id_v', 'manuf_id_m', 'cost_v', 'cost_m', 'MinOrV', 'MinOrM', 'uom_v', 'uom_m', 'mpn_mod', 'upc_mod']
    # STEP 1.1 find conflicts by equal mpn(upc), then equal manufacture_id
    sl_mas_upc = pd.merge(slave.dropna(subset=[upc_v_mod]), master, how='inner', left_on=upc_v_mod, right_on=upc_m_mod)
    sl_mas_mpn = pd.merge(slave.dropna(subset=[mpn_v_mod]), master, how='inner', left_on=mpn_v_mod, right_on=mpn_m_mod)
    if not sl_mas_upc.empty:
        sl_mas_upc.loc[:,'key'] = 'upc'
    if not sl_mas_mpn.empty:
        sl_mas_mpn.loc[:,'key'] = 'mpn'

    sl_mas = pd.concat([sl_mas_mpn, sl_mas_upc], ignore_index=True)
    if sl_mas.empty:
        return sl_mas

    # calculate ratio price: ratio_rp - retail price, ratio_dkp - DKH price, ratio_cp - cost price
    sl_mas.loc[(sl_mas[['price_v', 'ret_price_m']].notna().all(axis=1)) & (sl_mas[['price_v', 'ret_price_m']] != 0).all(axis=1), 'ratio_rp'] = abs(sl_mas['price_v'] - sl_mas['ret_price_m']) / sl_mas[['price_v', 'ret_price_m']].max(axis=1)
    sl_mas.loc[(sl_mas[['price_v', 'price_m']].notna().all(axis=1)) & (sl_mas[['price_v', 'price_m']] !=0).all(axis=1), 'ratio_dkp'] = abs(sl_mas['price_v'] - sl_mas['price_m']) / sl_mas[['price_v', 'price_m']].max(axis=1)
    sl_mas.loc[(sl_mas[['cost_v', 'cost_m']].notna().all(axis=1)) & (sl_mas[['cost_v', 'cost_m']] !=0).all(axis=1), 'ratio_cp'] = abs(sl_mas['cost_v'] - sl_mas['cost_m']) / sl_mas[['cost_v', 'cost_m']].max(axis=1)


    #set type=1 for equal manufacturers or brands
    normalize_name(sl_mas, name_columns=['raw_manuf_v', 'raw_brand_v', 'manuf_v', 'brand_v', 'raw_manuf_m', 'raw_brand_m', 'manuf_m', 'brand_m'], suffix='_cl')

    names_v = [i for i in sl_mas.columns if 'v_cl' in i]
    names_m = [i for i in sl_mas.columns if 'm_cl' in i]

    index_match = []
    for name_v in names_v:
        for name_m in names_m:
            index_match.extend(sl_mas[sl_mas[name_v] == sl_mas[name_m]].index)
    sl_mas.loc[index_match, 'type'] = 1
    # sl_mas.loc[(sl_mas['manuf_id_v'] == sl_mas['manuf_id_m'])|(sl_mas['brand_id_v'] == sl_mas['brand_id_m']), 'type'] = 1

    #set type=11 for equal manufacturers and something similar prices, it's means very reliable match  
    temp_filter = sl_mas.loc[sl_mas['type'] == 1, ['ratio_rp', 'ratio_dkp', 'ratio_cp']] <= ratio_prise_manuf
    temp_filter = temp_filter.any(axis=1)
    temp_filter = temp_filter[temp_filter == True]                              
    sl_mas.loc[temp_filter.index, 'type'] = 11
    sl_mas.loc[(sl_mas['type'] == 1) & (sl_mas['uom_v'] == sl_mas['uom_m']), 'type'] = 11

    #foget why i do it, maybe wrong? maybe another pd.version compare pd.NA > 1 = pd.NA? now it is False, so set not exist now
    #set type=12 for products was finded like compaunds and exist master products(type=11)
    # sl_mas.loc[(sl_mas['prod_id_v'].isin(sl_mas.loc[temp_filter.index, 'prod_id_v']) & (sl_mas['type']!=11).fillna(True)), 'type'] = 12
    # sl_mas.loc[~(sl_mas['prod_id_v'].isin(sl_mas.loc[sl_mas['type']==11, 'prod_id_v']) & sl_mas['type']!=11), 'type'] = 12

    #set type=10 for equal manufacturers and not similar prices, it's means matchinig like pack and each
    temp_filter = sl_mas.loc[sl_mas['type'] == 1, ['ratio_rp', 'ratio_dkp', 'ratio_cp']] > ratio_prise_manuf
    temp_filter = temp_filter.any(axis=1)# chek ANY value, because compare NaN and numeric is False
    temp_filter = temp_filter[temp_filter == True]                              
    sl_mas.loc[temp_filter.index, 'type'] = 10

    #set type=12 from other(type is None)  for mpn_mod_v=mpn_mod_m and upc_v=upc_m
    temp_filter = sl_mas.loc[(sl_mas['type'].isna()) & (sl_mas['key']=='mpn') & (sl_mas[upc_v_mod]==sl_mas[upc_m_mod])]
    sl_mas.loc[temp_filter.index, 'type'] = 12
    temp_filter = sl_mas.loc[(sl_mas['type'].isna()) & (sl_mas['key']=='upc') & (sl_mas[mpn_v_mod]==sl_mas[mpn_m_mod])]
    sl_mas.loc[temp_filter.index, 'type'] = 12

    # STEP 2 find conflicts by equal mpn, then remain row with differense between price_slave and price_master no more 20%
    sl_mas_long_mpn = filter_short_numeric_mpn(sl_mas[sl_mas['type'].isna() & (sl_mas['key'] == 'mpn')], name_col=mpn_v_mod, lenght=4, lenght_num=6)
    #set type=2 for not equal manufacturers and similar prices
    temp_filter = sl_mas_long_mpn.loc[:, ['ratio_rp', 'ratio_dkp', 'ratio_cp']] <= ratio_prise
    temp_filter = temp_filter.any(axis=1)
    temp_filter = temp_filter[temp_filter == True]                              
    sl_mas.loc[temp_filter.index, 'type'] = 2
    #same for equal only upc
    temp_filter = sl_mas.loc[sl_mas['type'].isna() & (sl_mas[upc_v_mod] == sl_mas[upc_m_mod]), ['ratio_rp', 'ratio_dkp', 'ratio_cp']] <= ratio_prise_manuf
    temp_filter = temp_filter.any(axis=1)
    temp_filter = temp_filter[temp_filter == True]
    sl_mas.loc[temp_filter.index, 'type'] = 21

    #set type=20 for not equal manufacturers and not similar prices
    # temp_filter = sl_mas.loc[sl_mas['type'].isna(), ['ratio_rp', 'ratio_dkp', 'ratio_cp']] > ratio_prise
    # temp_filter = temp_filter.any(axis=1)# check ANY value, because compare NaN and numeric is False
    # temp_filter = temp_filter[temp_filter == True]
    # sl_mas.loc[temp_filter.index, 'type'] = 20

    return sl_mas#.drop_duplicates(subset=['prod_id_v','prod_id_m', 'type']).convert_dtypes()

def creat_mapping_conflicts(slave, master, ratio_prise=0.2, ratio_prise_manuf=0.5, trust2='check_existing', source=None):
    """matchining mpn, which was lowly modified and this stage mark sa 'trust'=1
    matchining mpn, which was highly modified and this stage mark sa 'trust'=2
    """
    help1 = """ helper column 'trust': trust2='check_existing'('expand')
                matching mpn, which was lowly modified and this stage mark sa 'trust'=1
                matching mpn, which was highly modified and this stage mark sa 'trust'=2
            """
    help2 = """ helper column 'type':
                type=1  for equal manufacturers
                type=11 for equal manufacturers and something similar prices, it's means very reliable match
                type=12  will be set from type=1 for mpn_mod_v=mpn_mod_m and upc_v=upc_m
                type=10 for equal manufacturers and not similar prices, it's means matching like pack and each like compounds
                type=2  for not equal manufacturers and similar prices
                type=20 for not equal manufacturers and not similar prices
                type=21 for equal only upc
            """
    help3 = """ helper column 'key':
                type join vendor and master on upc or mpn
            """

    print(help1, help2, help3, sep='\n')

    if trust2 != 'check_existing' and  trust2 != 'expand':
        return print('Wrong choise parametr trust2, only: check_existing or expand')

    # STEP 1 modification mpn with a high level of reliability
    master = master.loc[master['ven_id']!=vendor_id].copy()

    normalize_upc(slave,  'upc_v', upc_v_mod)
    normalize_upc(master, 'upc_m', upc_m_mod)

    slave[mpn_v_mod] = slave['mpn_v'].apply(replace_pattern, args=('low',))
    master[mpn_m_mod] = master['mpn_m'].apply(replace_pattern, args=('low',))
    
    result_1 = find_conflicts(slave, master, ratio_prise, ratio_prise_manuf)
    #for debug
    # display(result_1)
    if not result_1.empty:
        result_1.loc[:, 'trust'] = 1
        
    # STEP 2 modification mpn with a low level of reliability
    if trust2 == 'check_existing':
        slave = slave[~slave['prod_id_v'].isin(result_1['prod_id_v'])].copy()
        slave[mpn_v_mod] = slave['mpn_v'].apply(replace_pattern, args=('high',))

    elif trust2 == 'expand':
        slave[mpn_v_mod] = slave['mpn_v'].apply(replace_pattern, args=('high',))

    master[mpn_m_mod] = master['mpn_m'].apply(replace_pattern, args=('high',))
    result_2 = find_conflicts(slave, master, ratio_prise, ratio_prise_manuf)

    if not result_2.empty:
        result_2.loc[:, 'trust'] = 2

    if result_1.empty and result_2.empty:
        print('Conflicts not found')
        return pd.DataFrame()


    # compare DK with DK
    if source == 'DK':
        cols = ['key','trust', 'type', 'prod_id_v', 'prod_id_m', 'mpn_v', 'mpn_m', 'upc_v', 'upc_m', 'manuf_v', 'raw_manuf_v', 'brand_v', 'raw_brand_v', 'manuf_m', 'brand_m', 'raw_manuf_m', 'raw_brand_m', 'ratio_rp', 'ratio_dkp', 'ratio_cp',
                'price_v', 'ret_price_m', 'price_m', 'title_v', 'title_m', 'cost_v', 'cost_m', 'MinOrV', 'MinOrM', 'uom_v', 'uom_m', 'manuf_id_v', 'manuf_id_m',  'brand_id_v', 'brand_id_m', mpn_v_mod]
    # compare VM with DK
    else:
        cols = ['key','trust', 'type', 'item_code_v', 'prod_id_v', 'prod_id_m', 'mpn_v', 'mpn_m', 'upc_v', 'upc_m', 'manuf_v', 'raw_manuf_v', 'brand_v', 'raw_brand_v', 'manuf_m', 'brand_m', 'raw_manuf_m', 'raw_brand_m', 'ratio_rp', 'ratio_dkp', 'ratio_cp',
            'price_v', 'ret_price_m', 'price_m', 'title_v', 'title_m', 'cost_v', 'cost_m', 'MinOrV', 'MinOrM', 'uom_v', 'uom_m', 'manuf_id_v', 'manuf_id_m', 'raw_manuf_id_v', 'brand_id_v', 'brand_id_m', 'raw_brand_id_v', mpn_v_mod, 'item_m', 'ven_id']


    result_conf = pd.concat([result_1, result_2], ignore_index=True).loc[:, cols]
    # exist_vendor_products = result_conf.loc[result_conf['ven_id']==127]

    #block for exclude duplicates with prioritet by type
    result_conf.loc[result_conf['type']==11, 'temp'] = 1
    result_conf.loc[result_conf['type']==12, 'temp'] = 2
    result_conf.loc[result_conf['type']==10, 'temp'] = 3
    result_conf.loc[result_conf['type']==1, 'temp'] = 4
    result_conf.loc[result_conf['type']==2, 'temp'] = 5
    result_conf.loc[result_conf['type']==21, 'temp'] = 6

    result_conf.sort_values('temp', inplace=True)
    result_conf.drop_duplicates(subset=['prod_id_v','prod_id_m'], keep='first', inplace=True)
    result_conf.drop(columns='temp', inplace=True)

    result_conf.insert(2, 'master', pd.NA)

    return result_conf#, exist_vendor_products.loc[:, cols]

def equal_manuf(df, what_do):
    print(pd.Timestamp.now())

    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params_test3}")
    table = 'equal_manuf_matching_products'

    if what_do == 'read':
        df_equal_manuf = pd.read_sql(f"SELECT * FROM {table}", con=engine).drop_duplicates()
        engine.dispose()
        return df_equal_manuf

    elif what_do == 'add':
        df.drop_duplicates().to_sql(table, con=engine, schema='dbo', if_exists='append', index=False)

    elif what_do == 'creat_new':
        df.drop_duplicates().to_sql(table, con=engine, schema='dbo', if_exists='replace', index=False)

    else:
        print('Wrong choise what_do')

    engine.dispose()
    return

def reconflict(df, ratio_prise=0.3, ratio_prise_manuf=0.7):
    """in df looks for manuf of products from equal_manuf_from_prod, this pair of manuf is configured as the same manuf and
    all products with this pair manuf type=1 and pair of manuf write to csv
    in df looks for products from equal_prod, this pair of products became equal - type=1
    """

    help = """ added to helper column 'type':
                type=3  (manual re-conflict) equal products for equivalence
                type=31 (manual re-conflict) for not equal manufacturers, not similar prices, Equivalents(key 'mpn':mpn_v = mpn_m AND upc_v=upc_m; key 'upc': only upc_v=upc_m)
                type=0  (manual re-conflict) not equal products
                type=77  (manual re-conflict) already raise matching conflicts ORGILL

                type=777 is temp label, which will be re-write to 10 or 11, and it not re-write which both ratio Nan
            """
    print(help)

    df = df.copy()
#     df_equal_manuf = pd.DataFrame(columns=['manuf_id_v', 'manuf_id_m', 'manuf_v', 'manuf_m'])
    try:
        df_equal_manuf = equal_manuf(None, what_do='read')
    except:
        df_equal_manuf = pd.DataFrame(columns=['manuf_id_v', 'manuf_id_m', 'manuf_v', 'manuf_m'])
        
    for products in equal_manuf_from_prod:
        manufs = df.loc[(df['prod_id_v']==products[0])&(df['prod_id_m']==products[1]), ['manuf_id_v', 'manuf_id_m', 'manuf_v', 'manuf_m']]
        if manufs.empty:
            continue
        df_equal_manuf = pd.concat([df_equal_manuf, manufs])
    
    df_equal_manuf.drop_duplicates(inplace=True)
    equal_manuf(df_equal_manuf, what_do='creat_new')

    label_eq_manuf_brand = 777 # temp label, which will be re-write to 10 or 11, and it not re-write which both ratio Nan
    for manufs in df_equal_manuf.itertuples():
        # df.loc[(df['manuf_id_v']==manufs.manuf_id_v) & (df['manuf_id_m']==manufs.manuf_id_m) & (df[['ratio_rp', 'ratio_dkp', 'ratio_cp']] <= ratio_prise_manuf).any(axis=1), 'type'] = 11 # direct verification
        # df.loc[(df['manuf_id_v']==manufs.manuf_id_m) & (df['manuf_id_m']==manufs.manuf_id_v) & (df[['ratio_rp', 'ratio_dkp', 'ratio_cp']] <= ratio_prise_manuf).any(axis=1), 'type'] = 11 # cross verification
        # #label for find pack
        # df.loc[(df['manuf_id_v']==manufs.manuf_id_v) & (df['manuf_id_m']==manufs.manuf_id_m) & (df[['ratio_rp', 'ratio_dkp', 'ratio_cp']] > ratio_prise_manuf).any(axis=1), 'type'] = 10 # direct verification
        # df.loc[(df['manuf_id_v']==manufs.manuf_id_m) & (df['manuf_id_m']==manufs.manuf_id_v) & (df[['ratio_rp', 'ratio_dkp', 'ratio_cp']] > ratio_prise_manuf).any(axis=1), 'type'] = 10 # cross verification
        df.loc[(df['manuf_id_v']==manufs.manuf_id_v) & (df['manuf_id_m']==manufs.manuf_id_m), 'type'] = label_eq_manuf_brand # direct verification
        df.loc[(df['manuf_id_v']==manufs.manuf_id_m) & (df['manuf_id_m']==manufs.manuf_id_v), 'type'] = label_eq_manuf_brand # cross verification
        #label for find pack
        df.loc[(df['manuf_id_v']==manufs.manuf_id_v) & (df['manuf_id_m']==manufs.manuf_id_m), 'type'] = label_eq_manuf_brand # direct verification
        df.loc[(df['manuf_id_v']==manufs.manuf_id_m) & (df['manuf_id_m']==manufs.manuf_id_v), 'type'] = label_eq_manuf_brand # cross verification

    #products_vendor hasn't mapping manuf, but its equal manufs
    df_manuf_mapping = pd.DataFrame()
    df_brand_mapping = pd.DataFrame()
    for products in raw_equal_manuf_from_prod:
        manuf_mapping = df.loc[(df['prod_id_v']==products[0])&(df['prod_id_m']==products[1])&(df['manuf_id_v'].isna()), ['raw_manuf_id_v', 'manuf_id_m', 'raw_manuf_v', 'manuf_m', 'manuf_id_v', 'manuf_v']]
        brand_mapping = df.loc[(df['prod_id_v']==products[0])&(df['prod_id_m']==products[1])&(df['brand_id_v'].isna()), ['raw_brand_id_v', 'brand_id_m', 'raw_brand_v', 'brand_m', 'brand_id_v', 'brand_v']]

        if not manuf_mapping.empty:# and pd.isna(manuf_mapping['manuf_id_v']):
            df_manuf_mapping = pd.concat([df_manuf_mapping, manuf_mapping], ignore_index=True)
        if not brand_mapping.empty:# and pd.isna(brand_mapping['brand_id_v']):
            df_brand_mapping = pd.concat([df_brand_mapping, brand_mapping], ignore_index=True)

    df_manuf_mapping.drop_duplicates(inplace=True)
    df_brand_mapping.drop_duplicates(inplace=True)

    for manufs in df_manuf_mapping.itertuples():
        df.loc[(df['raw_manuf_id_v']==manufs.raw_manuf_id_v) & (df['manuf_id_m']==manufs.manuf_id_m), 'type'] = label_eq_manuf_brand # direct verification
    for manufs in df_brand_mapping.itertuples():
        df.loc[(df['raw_brand_id_v']==manufs.raw_brand_id_v) & (df['brand_id_m']==manufs.brand_id_m), 'type'] = label_eq_manuf_brand # direct verification

    df.loc[(df['type']==label_eq_manuf_brand) & (df[['ratio_rp', 'ratio_dkp', 'ratio_cp']] <= ratio_prise_manuf).any(axis=1), 'type'] = 11 # direct verification
    df.loc[(df['type']==label_eq_manuf_brand) & (df[['ratio_rp', 'ratio_dkp', 'ratio_cp']] > ratio_prise_manuf).any(axis=1), 'type'] = 10 # direct verification
    df.loc[(df['type']==label_eq_manuf_brand) & (df['uom_v'] == df['uom_m']), 'type'] = 11 # direct verification


    #differ mfg brand, but equal products, its mean Equivalents SET type = 3
    for products in equal_prod:
        df.loc[(df['prod_id_v']==products[0])&(df['prod_id_m']==products[1]), 'type'] = 3

    #products_vendor hasn't mapping manuf, but its not equal manufs and equal products, its mean Equivalents  SET type = 3
    df_equivalents_manuf = pd.DataFrame()
    for products in raw_manuf_from_equivalents_prod:
        equivalents_manuf = df.loc[(df['prod_id_v']==products[0])&(df['prod_id_m']==products[1]), ['raw_manuf_id_v', 'manuf_id_m', 'raw_manuf_v', 'manuf_m']]
        if equivalents_manuf.empty:
            continue
        df_equivalents_manuf = pd.concat([df_equivalents_manuf, equivalents_manuf])
        df_equivalents_manuf.drop_duplicates(inplace=True)

    for manufs in df_equivalents_manuf.itertuples():
        df.loc[(df['raw_manuf_id_v']==manufs.raw_manuf_id_v)&(df['manuf_id_m']==manufs.manuf_id_m), 'type'] = 3 # direct verification

    #not equal prods
    for products in not_same_prod:
        df.loc[(df['prod_id_v']==products[0])&(df['prod_id_m']==products[1]), ['type', 'master']] = (0, False)

    #start only for orgill
    # temp_orgill_alregy_raise_matchining = pd.DataFrame()
    # temp_orgill_alregy_raise_matchining = pd.concat([temp_orgill_alregy_raise_matchining, pd.read_csv('equal_prod_irwing.zip')], ignore_index=True)
    #
    # for products in temp_orgill_alregy_raise_matchining.itertuples():
    #     df.loc[(df['prod_id_v']==products.prod_id_v)&(df['prod_id_m']==products.prod_id_m), 'type'] = 77
    #end only orgill

    #Equivalents(key 'mpn':mpn_v = mpn_m AND upc_v=upc_m; key 'upc': only upc_v=upc_m)
    # df.loc[((df['type']==20) | (df['type'].isna())) & (df['upc_v_mod'] == df['upc_m_mod']), 'type'] = 31

    #calc alfabeta in mpn_mod for filtering later
    df['cnt_abc'] = df.loc[df[mpn_v_mod].notna(), mpn_v_mod].apply(lambda row: len([i for i in row if i.isalpha()]))
    df.drop_duplicates(subset=['prod_id_v','prod_id_m', 'type'], keep='first', inplace=True)

    #drop item from multiconflicts, rest only type with high priority like type 11
    reliable_list_id = df.loc[df['type'] == 11, 'prod_id_v'].unique()
    remove_items = df.loc[(df['type'] != 11) & df['type'].notna() & df['prod_id_v'].isin(reliable_list_id)].index
    df.drop(remove_items, inplace=True)


    return df, df_manuf_mapping, df_brand_mapping

def extract_single_feature(df, column, count=1, compare='='):
    df_loc = df.copy()
    calc_feat = pd.DataFrame(df_loc.loc[:, column].value_counts()).reset_index()

    if compare == '>':
        flt_calc_feat = calc_feat.loc[calc_feat[column] > count, 'index']
    elif compare == '=':
        flt_calc_feat = calc_feat.loc[calc_feat[column] == count, 'index']
    else:
        return print('wrong compare')

    flt_df = df_loc[df_loc[column].isin(flt_calc_feat)].sort_values(column)

    return flt_df.sort_values(column)



In [44]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

C:\Users\duser05\AppData\Local\Temp\ipykernel_19584\4085591814.py:10: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(locals().items())), key= lambda x: -x[1])[:10]:
C:\Users\duser05\AppData\Local\Temp\ipykernel_19584\4085591814.py:10: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(locals().items())), key= lambda x: -x[1])[:10]:


                      prod_DKH:  3.9 GiB
                           _17: 13.9 MiB
                    cand_confl: 10.9 MiB
    all_mapped_products_vendor:  9.1 MiB
                   prod_vendor:  4.0 MiB
                 cand_confl_re:  3.7 MiB
                           _18:  3.7 MiB
                           _20:  3.7 MiB
                     clearprod:  3.7 MiB
             cand_confl_re_flt:  3.7 MiB


In [45]:
del obj

NameError: name 'obj' is not defined

#### Select master products

In [6]:
print(pd.Timestamp.now())

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params_DK}")

query = """
		SELECT DISTINCT p.product_ID AS prod_id_m, p.product_name as title_m
                ,pvm1.value AS mpn_m
                --,pvm2.value AS item_m
                --,pvm3.value AS sku_m
                ,trim(COALESCE(pvm2.value, pvm3.value, p.product_item_code)) AS item_m
                ,trim(COALESCE(pvm4.value, p.UPC_code)) AS upc_m
                ,m.manufacturer_ID AS manuf_id_m
                ,m.name AS manuf_m
                ,vp.price AS cost_m, p.retail_price AS ret_price_m, p.price AS price_m
                ,COALESCE(vp.units_from, p.min_order) AS MinOrM
                ,p.product_hold AS "hold"
                ,p.brand_id AS brand_id_m
                ,pb.name AS brand_m
                ,um.unit_measure AS uom_m
                ,m1.name AS raw_manuf_m, b1.name AS raw_brand_m
                ,pv.vendor_id AS ven_id
        FROM DKHardware_NEW.dbo.products p
        JOIN DKHardware_NEW.dbo.manufacturers m ON m.manufacturer_ID = p.manufacturer_ID
        JOIN DKHardware_NEW.dbo.products_brands pb ON pb.id = p.brand_id
        JOIN DKHardware_NEW.dbo.products_vendors pv ON pv.product_id = p.product_ID AND pv.vendor_id NOT IN (1,6,9,27,28,40)
        LEFT JOIN DKHardware_NEW.dbo.products_vendors_meta pvm1 ON pvm1.product_vendor_id = pv.id AND pvm1.code = 'mpn'
        LEFT JOIN DKHardware_NEW.dbo.products_vendors_meta pvm2 ON pvm2.product_vendor_id = pv.id AND pvm2.code = 'itemCode'
        LEFT JOIN DKHardware_NEW.dbo.products_vendors_meta pvm3 ON pvm3.product_vendor_id = pv.id AND pvm3.code = 'SKU'
        LEFT JOIN DKHardware_NEW.dbo.products_vendors_meta pvm4 ON pvm4.product_vendor_id = pv.id AND pvm4.code = 'UPC'
        LEFT JOIN DKHardware_NEW.dbo.products_vendors_units pvu ON  pvu.product_vendor_id = pv.id
        LEFT JOIN DKHardware_NEW.dbo.unit_measure um ON um.unit_ID = pvu.unit_of_measure_id
        LEFT JOIN DKHardware_NEW.dbo.vendors_prices vp ON vp.product_vendor_id = pv.id
        LEFT JOIN VendorMigrator.dbo.products p2 ON p2.master_product_id = pv.product_id AND pv.vendor_id = p2.vendor_id
        LEFT JOIN VendorMigrator.dbo.products_versions pv2 ON pv2.product_id = p2.id AND pv2.version =
            (
            SELECT MAX(pv3.version)
            FROM VendorMigrator.dbo.products_versions pv3
            WHERE pv3.product_id = pv2.product_id
            )
        LEFT JOIN VendorMigrator.dbo.products_fields pf ON pv2.id = pf.product_version_id
        LEFT JOIN VendorMigrator.dbo.manufacturers m1 ON pf.manufacturer_id = m1.id
        LEFT JOIN VendorMigrator.dbo.brands b1 ON b1.id = pf.brand_id
        WHERE (p.product_hold = 0 OR p.product_hidden = 0)

        """
query_uom = """
        SELECT *
        FROM unit_measure
        """

prod_DKH = pd.read_sql(query, con=engine)
unit_measure_DKH = pd.read_sql(query_uom, con=engine)
engine.dispose()
prod_DKH.to_csv('prod_DKH.zip', index=False)
prod_DKH.info(memory_usage='deep')
print(pd.Timestamp.now())

2023-10-26 18:04:00.793226
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6248580 entries, 0 to 6248579
Data columns (total 18 columns):
 #   Column       Dtype  
---  ------       -----  
 0   prod_id_m    int64  
 1   title_m      object 
 2   mpn_m        object 
 3   item_m       object 
 4   upc_m        object 
 5   manuf_id_m   int64  
 6   manuf_m      object 
 7   cost_m       float64
 8   ret_price_m  float64
 9   price_m      float64
 10  MinOrM       float64
 11  hold         bool   
 12  brand_id_m   int64  
 13  brand_m      object 
 14  uom_m        object 
 15  raw_manuf_m  object 
 16  raw_brand_m  object 
 17  ven_id       int64  
dtypes: bool(1), float64(4), int64(4), object(9)
memory usage: 3.9 GB
2023-10-26 18:43:12.525262


#### Select vendor products from DK Hardware

In [8]:
print(pd.Timestamp.now())

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params_DK}")

vendor_id = 127

query = f"""
            SELECT DISTINCT p.product_ID AS prod_id_v, p.product_name as title_v
                ,pvm1.value AS mpn_v
                --,pvm2.value AS item_v
                --,pvm3.value AS sku_V
                ,trim(COALESCE(pvm2.value, pvm3.value, p.product_item_code)) AS item_v
                ,trim(COALESCE(pvm4.value, p.UPC_code)) AS upc_v
                ,m.manufacturer_ID AS manuf_id_v
                ,m.name AS manuf_v
                ,vp.price AS cost_v, p.retail_price AS ret_price_v, p.price AS price_v
                ,COALESCE(vp.units_from, p.min_order) AS MinOrV
                ,p.product_hold AS "hold"
                ,p.brand_id AS brand_id_v
                ,pb.name AS brand_v
                ,um.unit_measure AS uom_v
                ,m1.name AS raw_manuf_v, b1.name AS raw_brand_v
                ,pv.vendor_id AS ven_id
        FROM DKHardware_NEW.dbo.products p
        JOIN DKHardware_NEW.dbo.manufacturers m ON m.manufacturer_ID = p.manufacturer_ID
        JOIN DKHardware_NEW.dbo.products_brands pb ON pb.id = p.brand_id
        JOIN DKHardware_NEW.dbo.products_vendors pv ON pv.product_id = p.product_ID AND p.vendor_ID = {vendor_id}
        LEFT JOIN DKHardware_NEW.dbo.products_vendors_meta pvm1 ON pvm1.product_vendor_id = pv.id AND pvm1.code = 'mpn'
        LEFT JOIN DKHardware_NEW.dbo.products_vendors_meta pvm2 ON pvm2.product_vendor_id = pv.id AND pvm2.code = 'itemCode'
        LEFT JOIN DKHardware_NEW.dbo.products_vendors_meta pvm3 ON pvm3.product_vendor_id = pv.id AND pvm3.code = 'SKU'
        LEFT JOIN DKHardware_NEW.dbo.products_vendors_meta pvm4 ON pvm4.product_vendor_id = pv.id AND pvm4.code = 'UPC'
        LEFT JOIN DKHardware_NEW.dbo.products_vendors_units pvu ON  pvu.product_vendor_id = pv.id
        LEFT JOIN DKHardware_NEW.dbo.unit_measure um ON um.unit_ID = pvu.unit_of_measure_id
        LEFT JOIN DKHardware_NEW.dbo.vendors_prices vp ON vp.product_vendor_id = pv.id
        LEFT JOIN VendorMigrator.dbo.products p2 ON p2.master_product_id = pv.product_id AND pv.vendor_id = p2.vendor_id
        LEFT JOIN VendorMigrator.dbo.products_versions pv2 ON pv2.product_id = p2.id AND pv2.version =
            (
            SELECT MAX(pv3.version)
            FROM VendorMigrator.dbo.products_versions pv3
            WHERE pv3.product_id = pv2.product_id
            )
        LEFT JOIN VendorMigrator.dbo.products_fields pf ON pv2.id = pf.product_version_id
        LEFT JOIN VendorMigrator.dbo.manufacturers m1 ON pf.manufacturer_id = m1.id
        LEFT JOIN VendorMigrator.dbo.brands b1 ON b1.id = pf.brand_id
        WHERE (p.product_hold = 0 OR p.product_hidden = 0)
        	AND p.product_ID IN
        		(
        		SELECT DISTINCT p3.product_ID
        		FROM products p3
        		JOIN products_vendors pv4 ON pv4.product_id = p3.product_ID AND pv4.vendor_id = {vendor_id}
        		LEFT JOIN products_vendors pv5 ON pv5.product_id = p3.product_ID AND pv5.vendor_id != {vendor_id}
        		WHERE pv5.id is null and p3.manufacturer_ID = 32986
        		)
        """

prod_vendor_DKH = pd.read_sql(query, con=engine)
engine.dispose()
prod_vendor_DKH.to_csv('prod_vendor_DKH.zip', index=False)
prod_vendor_DKH.info(memory_usage='deep')
print(pd.Timestamp.now())

2023-10-11 16:34:54.200713
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   prod_id_v    14 non-null     int64  
 1   title_v      14 non-null     object 
 2   mpn_v        14 non-null     object 
 3   item_v       14 non-null     object 
 4   upc_v        14 non-null     object 
 5   manuf_id_v   14 non-null     int64  
 6   manuf_v      14 non-null     object 
 7   cost_v       14 non-null     float64
 8   ret_price_v  14 non-null     float64
 9   price_v      14 non-null     float64
 10  MinOrV       14 non-null     float64
 11  hold         14 non-null     bool   
 12  brand_id_v   14 non-null     int64  
 13  brand_v      14 non-null     object 
 14  uom_v        14 non-null     object 
 15  raw_manuf_v  14 non-null     object 
 16  raw_brand_v  14 non-null     object 
 17  ven_id       14 non-null     int64  
dtypes: bool(1), float64(4), i

#### Select vendor products

In [26]:
%%time
print(pd.Timestamp.now())

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params_VM}")
vendor_id = 127
prefix = "'org_'"
query1 = f"""
             SELECT  p.id AS prod_id_v
            		, pf.name as title_v
            		, pf.manuf_code AS mpn_v
            		, pf.upc_code AS upc_v
            		, dkm.manufacturer_id AS manuf_id_v
            		, dkm.name AS manuf_v
            		, replace(p.vendor_product_code, {prefix}, '') AS item_code_v
            		, m.id as raw_manuf_id_v
            		, m.name AS raw_manuf_v
            		, b.id AS raw_brand_id_v
            		, b.name AS raw_brand_v
            		, pb.id AS brand_id_v
            		, pb.name AS brand_v
            FROM products p
            JOIN products_versions pv ON pv.product_id = p.id AND pv.version =
                (
                SELECT MAX(version)
                FROM products_versions pv2
                WHERE pv2.product_id = pv.product_id AND (pv2.version_status = 1 OR pv2.version_status IS NULL)
                )
            JOIN products_fields pf ON pv.id = pf.product_version_id
            JOIN manufacturers m ON pf.manufacturer_id = m.id
            JOIN brands b ON b.id = pf.brand_id
            LEFT JOIN manufacturers_to_master_map mtmm ON m.id = mtmm.manufacturer_id
            LEFT JOIN DKHardware_NEW.dbo.manufacturers dkm ON dkm.manufacturer_ID = mtmm.master_manufacturer_id
            LEFT JOIN brands_to_master_map btmm on btmm.brand_id = b.id
            LEFT JOIN DKHardware_NEW.dbo.products_brands pb ON pb.id = btmm.master_brand_id
            WHERE p.vendor_id = {vendor_id} and p.master_product_id IS NULL
        """

query2 = f"""
            SELECT *
            FROM products p
            WHERE p.vendor_id = {vendor_id} and p.master_product_id IS NOT NULL
        """

products_vendor = pd.read_sql(query1, con=engine)
all_mapped_products_vendor = pd.read_sql(query2, con=engine)
engine.dispose()
products_vendor.to_csv('products_vendor.zip', index=False)
products_vendor.info(memory_usage='deep')
print(pd.Timestamp.now())
products_vendor.loc[products_vendor['upc_v']=='000000000000', 'upc_v'] = pd.NA


2023-10-27 09:52:22.244958
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   prod_id_v       0 non-null      object
 1   title_v         0 non-null      object
 2   mpn_v           0 non-null      object
 3   upc_v           0 non-null      object
 4   manuf_id_v      0 non-null      object
 5   manuf_v         0 non-null      object
 6   item_code_v     0 non-null      object
 7   raw_manuf_id_v  0 non-null      object
 8   raw_manuf_v     0 non-null      object
 9   raw_brand_id_v  0 non-null      object
 10  raw_brand_v     0 non-null      object
 11  brand_id_v      0 non-null      object
 12  brand_v         0 non-null      object
dtypes: object(13)
memory usage: 0.0 bytes
2023-10-27 09:52:31.232764
CPU times: total: 547 ms
Wall time: 8.99 s


#### Select vendor prices

In [8]:
print(pd.Timestamp.now())

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params_vendor_orgill}")

query = """
        SELECT p.product_item_code, p.units_from AS MinOrV, p.cost AS cost_v, p.price AS price_v, p.unit_measure AS uom_v
        FROM prices p;
        """

prices_vendor_old = pd.read_sql(query, con=engine)
engine.dispose()
prices_vendor_old.to_csv('prices_vendor_old.zip', index=False)
prices_vendor_old.info(memory_usage='deep')
print(pd.Timestamp.now())

2023-10-27 09:34:12.353945
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1166 entries, 0 to 1165
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product_item_code  1166 non-null   object 
 1   MinOrV             1166 non-null   int64  
 2   cost_v             1166 non-null   float64
 3   price_v            1166 non-null   float64
 4   uom_v              1166 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 167.6 KB
2023-10-27 09:34:16.315562


#### Read tables from local csv

In [124]:
# prod_DKH = pd.read_csv('prod_DKH.zip', dtype={'upc_m':'str', 'mpn_m':'str', 'cost_m':np.float64, 'ret_price_m':np.float64 ,'price_m':np.float64})
# products_vendor = pd.read_csv('products_vendor.zip', dtype={'upc_v':'str','mpn_v':'str', 'prod_id_v':'int32', 'manuf_id_v':'int32', 'item_code_v':'str', 'raw_manuf_id_v':'int32', 'raw_brand_id_v':'int32', 'brand_id_v':'int32'})
# prices_vendor = pd.read_csv('prices_vendor.zip')
# prices_vendor_old = pd.read_csv('prices_vendor_old.zip', dtype={'upc_v':'str','mpn_v':'str', 'prod_id_v':'int32', 'product_item_code':'str', 'manuf_id_v':'int32', 'raw_manuf_id_v':'int32', 'raw_brand_id_v':'int32', 'brand_id_v':'int32'})
# # prices_vendor_new = pd.read_csv('prices_vendor_new.zip')
# products_vendor.info()
# prices_vendor_old.info()

In [9]:
# prices_vendor = pd.concat([prices_vendor_old, prices_vendor_new], ignore_index=True)
prices_vendor = pd.concat([prices_vendor_old], ignore_index=True)
prices_vendor.drop_duplicates(subset=['product_item_code'], keep='last', inplace=True, ignore_index=True)
#convert uom_abbr to uom_name
prices_vendor = pd.merge(prices_vendor, unit_measure_DKH, how='inner', left_on='uom_v', right_on='uom', validate='m:1')
prices_vendor['uom_v'] = prices_vendor['unit_measure']
prices_vendor.loc[:,prices_vendor_old.columns]


prod_vendor = pd.merge(products_vendor, prices_vendor, how='left', left_on='item_code_v', right_on='product_item_code', validate='m:1')
prod_vendor.drop(['product_item_code'], axis=1, inplace=True )

print(f"size prod_vendor {prod_vendor.shape}")
display(prod_vendor.sample())
print(f"size prod_DKH {prod_DKH.shape}")
display(prod_DKH.sample())

size prod_vendor (10, 22)


,prod_id_v,title_v,mpn_v,upc_v,manuf_id_v,manuf_v,item_code_v,raw_manuf_id_v,raw_manuf_v,raw_brand_id_v,...,brand_v,MinOrV,cost_v,price_v,uom_v,unit_measure,unit_ID,uom,variation_template,variation_plural_template
0,9821505,"PowerPlus Hardwood Floor Deep Cleaner Refill, ...",WM850018001,737025012434,2511,Bona,0032011,47225,BONA,18428,...,Bona,1,13.31,24.75,Each,Each,17,EA,each,pack of {0}


size prod_DKH (6248580, 18)


,prod_id_m,title_m,mpn_m,item_m,upc_m,manuf_id_m,manuf_m,cost_m,ret_price_m,price_m,MinOrM,hold,brand_id_m,brand_m,uom_m,raw_manuf_m,raw_brand_m,ven_id
1163278,2391400,Cylindrical Lock Satin Brass,28-10G26 GL 4,28-10G26 GL 4,None,216,Sargent,436.9,734.0,711.45,1.0,False,1835,Sargent,Each,Sargent,Sargent,44


In [10]:
print(pd.Timestamp.now())
# two step version find conflicts

cand_confl = creat_mapping_conflicts(prod_vendor, prod_DKH, ratio_prise=0.35, ratio_prise_manuf=0.5, trust2='check_existing')
# cand_confl = creat_mapping_conflicts(prod_vendor_DKH, prod_DKH, ratio_prise=0.35, ratio_prise_manuf=0.5, trust2='check_existing', source='DK')

# confl_exist_prod = cand_confl.loc[cand_confl['prod_id_m'].isin(prod_DKH.loc[prod_DKH['ven_id']==vendor_id, 'prod_id_m'])]
# new_products = products_vendor.loc[~products_vendor['prod_id_v'].isin(cand_confl['prod_id_v'])]
new_products = products_vendor.loc[~products_vendor['prod_id_v'].isin(cand_confl.loc[cand_confl['type'].notna(), 'prod_id_v'])]
print(pd.Timestamp.now())
print(cand_confl.shape)
print(f'New products {new_products.shape[0]}')
cand_confl['type'].value_counts(dropna=False)

2023-10-27 09:34:22.392397
 helper column 'trust': trust2='check_existing'('expand')
                matching mpn, which was lowly modified and this stage mark sa 'trust'=1
                matching mpn, which was highly modified and this stage mark sa 'trust'=2
            
 helper column 'type':
                type=1  for equal manufacturers
                type=11 for equal manufacturers and something similar prices, it's means very reliable match
                type=12  will be set from type=1 for mpn_mod_v=mpn_mod_m and upc_v=upc_m
                type=10 for equal manufacturers and not similar prices, it's means matching like pack and each like compounds
                type=2  for not equal manufacturers and similar prices
                type=20 for not equal manufacturers and not similar prices
                type=21 for equal only upc
            
 helper column 'key':
                type join vendor and master on upc or mpn
            
2023-10-27 09:36:39.848043
(15, 42)

11.0    9
NaN     5
2.0     1
Name: type, dtype: int64

In [11]:
new_products

,prod_id_v,title_v,mpn_v,upc_v,manuf_id_v,manuf_v,item_code_v,raw_manuf_id_v,raw_manuf_v,raw_brand_id_v,raw_brand_v,brand_id_v,brand_v


In [12]:
equal_manuf_from_prod = [(8730510,5643196), (8766941,3160965),(8674269,4516246),(8651036,5076962),(8710686,4120892),(8698418,5264047),(8647672,5764281),(8718540,3155158), (8716477,3360593),
                         (8762276,5261434), (8765830,4133787),(8763634,5735953),(8704557,5537956),(8660505,5545456),(8659024,5524284), (8697046,3936444),(8694528,5574577), (8744329,6543641),
                         (8666386,5255044), (8748465,5533870), (8742936,4541317),(8716742,5599997), (8653674,3157632), (8710304,71649), (8692399,3658670), (8749001,5724589), (8654909,5961911),
                         (8788611,661577), (9726052,4656273), (9802841,5554139), (9802705,5737147), (9802936,5645278), (9800897,5725245), (9802149,491174), (9802138,5573623), (9801020,492336),
                         (9800577,5582817), (9801811,695995), (9802941,5509119), (9800476,496113), (9802296,5607554), (9802140,482359), (9800606,5845772), (9800606,465044), (9800606,5525465),
                         (9802058,4629500), (9726089,3934481), (9801889,5590338), (9802615,5735105), (9802887,5961086),(9800895,5724383), (9800726,6629120), (9802130,2824475)] #orgill

#raw_equal_manuf_from_prod for products hasn't mapping manuf or brand
raw_equal_manuf_from_prod = []

raw_manuf_from_equivalents_prod = [(8661739,5120199),(8649905,4818347), (8724745,496189),(8714820,496556), (8673798,3656920), (8710470,3133244)]

#equal_prod for equal products vendors maybe differ or equal(havn't ratio prices)
equal_prod = [(8696280,4505341), (8708317,5600578), (8726269,5721481), (6634000,8728986), (8725336,3653473), (8677859,4225346)]

not_same_prod = [(8691977,3152050),(8735692,4075541),(8730617,3914932),(8765969,5074573),(8724464,5074573),(8662283, 5053855),(8713577, 5053856),(8727865, 5335057),(8729658, 5330091),(8751103, 5053856),(8759778, 5053855),
                 (8762973, 5166056), (8732868,6582993),(8716894,6581918)]

#add to list other list from file
cand_confl_re, maping_manuf, maping_brand = reconflict(cand_confl, ratio_prise=0.3, ratio_prise_manuf=0.5)
print(cand_confl_re.shape)

normalize_upc(cand_confl_re,  'upc_v', upc_v_mod)
normalize_upc(cand_confl_re,  'upc_m', upc_m_mod)
column_normalize = ['manuf_v','raw_manuf_v', 'manuf_m', 'raw_manuf_m', 'brand_v', 'raw_brand_v', 'brand_m', 'raw_brand_m']
normalize_name(cand_confl_re, name_columns=column_normalize, suffix = '_cl')

cand_confl_re.loc[(cand_confl_re[upc_v_mod] == cand_confl_re[upc_m_mod]) & cand_confl_re['type'].isna(), 'type'] = 2

if cand_confl_re.loc[cand_confl_re['type'].notna() & cand_confl_re['prod_id_m'].isin(all_mapped_products_vendor['master_product_id'])].empty:
    print("doubling masters will not finded. it's OK")
else:
    print('Warning! Doubling the masters will finded')
    conf_exist_myself = cand_confl_re.loc[cand_confl_re['prod_id_m'].isin(all_mapped_products_vendor['master_product_id'])].copy()
    cand_confl_re = cand_confl_re.loc[~cand_confl_re['prod_id_m'].isin(all_mapped_products_vendor['master_product_id'])].copy()
    display(conf_exist_myself)




#check list prod upper
# temp = pd.DataFrame()
# for v, m in _disable_raw_equal_manuf_from_prod:
#     temp = pd.concat([temp, cand_confl_re.loc[(cand_confl_re[['prod_id_v', 'prod_id_m']]==(v,m)).all(axis=1)]])
display(cand_confl_re['type'].value_counts(dropna=False))

 added to helper column 'type':
                type=3  (manual re-conflict) equal products for equivalence
                type=31 (manual re-conflict) for not equal manufacturers, not similar prices, Equivalents(key 'mpn':mpn_v = mpn_m AND upc_v=upc_m; key 'upc': only upc_v=upc_m)
                type=0  (manual re-conflict) not equal products
                type=77  (manual re-conflict) already raise matching conflicts ORGILL

                type=777 is temp label, which will be re-write to 10 or 11, and it not re-write which both ratio Nan
            
2023-10-27 09:36:50.491579
2023-10-27 09:36:53.707467
(15, 43)
Warning! Doubling the masters will finded


,key,trust,master,type,item_code_v,prod_id_v,prod_id_m,mpn_v,mpn_m,upc_v,...,upc_v_mod,upc_m_mod,manuf_v_cl,raw_manuf_v_cl,manuf_m_cl,raw_manuf_m_cl,brand_v_cl,raw_brand_v_cl,brand_m_cl,raw_brand_m_cl
0,mpn,1.0,<NA>,11.0,0032011,9821505,5547015,WM850018001,WM850018001,737025012434,...,737025012434,737025012434,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
25,upc,1.0,<NA>,11.0,0143958,9821508,4102222,817,917,037321009177,...,37321009177,37321009177,bonide,bonide,bonide,bonide,bonide,bonide,bonide,bonide
28,upc,1.0,<NA>,11.0,7489057,9821599,5540461,WM700018159,WM700018159,737025181598,...,737025181598,737025181598,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
30,upc,1.0,<NA>,11.0,7489073,9821600,5540462,WM700018172,WM700018172,737025181727,...,737025181727,737025181727,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
32,upc,1.0,<NA>,11.0,7489081,9821601,5541322,WM700051171,WM700051171,737025511548,...,737025511548,737025511548,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
34,upc,1.0,<NA>,11.0,7489099,9821602,5540464,WM700051184,WM700051184,737025511845,...,737025511845,737025511845,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
36,upc,1.0,<NA>,11.0,7489107,9821603,5529022,WM700051223,WM700051223,737025018764,...,737025018764,737025018764,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
38,upc,1.0,<NA>,11.0,7489115,9821604,5530118,WM700051224,WM700051224,737025018771,...,737025018771,737025018771,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
40,upc,1.0,<NA>,11.0,8850729,9821701,5539598,WT760051161,WT760051161,737025511616,...,737025511616,737025511616,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi


NaN    5
2.0    1
Name: type, dtype: int64

In [13]:
cand_confl_re[cand_confl_re['type']==2]

,key,trust,master,type,item_code_v,prod_id_v,prod_id_m,mpn_v,mpn_m,upc_v,...,upc_v_mod,upc_m_mod,manuf_v_cl,raw_manuf_v_cl,manuf_m_cl,raw_manuf_m_cl,brand_v_cl,raw_brand_v_cl,brand_m_cl,raw_brand_m_cl
13,mpn,1.0,<NA>,2.0,0384784,9821527,5022574,34-874T/M,34-874T/M,616314121555,...,616314121555,616314215551,boss mfg,boss mfg,pip,pip,boss,boss mfg,pip,pip


In [14]:
good_type = [11, 12]
cand_confl_re_flt = cand_confl_re[cand_confl_re['type'].isin(good_type) & (cand_confl_re['master']!=True)].drop_duplicates(subset=['prod_id_v', 'prod_id_m']).copy()
# cand_confl_re_flt = cand_confl_re[cand_confl_re['type'].notna() & (cand_confl_re['master']!=True)].drop_duplicates(subset=['prod_id_v', 'prod_id_m']).copy()

single_products_vendor = extract_single_feature(cand_confl_re_flt, 'prod_id_m', 1, '=') #('>','=')
single_matching = extract_single_feature(single_products_vendor, 'prod_id_v', 1, '=') #single product vendor to some master products
multi_matching = extract_single_feature(single_products_vendor, 'prod_id_v', 1, '>') #single product vendor to some master products
single_matching



,key,trust,master,type,item_code_v,prod_id_v,prod_id_m,mpn_v,mpn_m,upc_v,...,upc_v_mod,upc_m_mod,manuf_v_cl,raw_manuf_v_cl,manuf_m_cl,raw_manuf_m_cl,brand_v_cl,raw_brand_v_cl,brand_m_cl,raw_brand_m_cl


In [15]:
# conf_exist_myself

conf_exist_myself_flt = conf_exist_myself[conf_exist_myself['type'].isin(good_type) & (conf_exist_myself['master']!=True)].drop_duplicates(subset=['prod_id_v', 'prod_id_m']).copy()

single_products_vendor_to_itself_master = extract_single_feature(conf_exist_myself_flt, 'prod_id_m', 1, '=') #('>','=')
single_equivalents = extract_single_feature(single_products_vendor_to_itself_master, 'prod_id_v', 1, '=') #single product vendor to some master products
single_equivalents

,key,trust,master,type,item_code_v,prod_id_v,prod_id_m,mpn_v,mpn_m,upc_v,...,upc_v_mod,upc_m_mod,manuf_v_cl,raw_manuf_v_cl,manuf_m_cl,raw_manuf_m_cl,brand_v_cl,raw_brand_v_cl,brand_m_cl,raw_brand_m_cl
0,mpn,1.0,<NA>,11.0,0032011,9821505,5547015,WM850018001,WM850018001,737025012434,...,737025012434,737025012434,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
25,upc,1.0,<NA>,11.0,0143958,9821508,4102222,817,917,037321009177,...,37321009177,37321009177,bonide,bonide,bonide,bonide,bonide,bonide,bonide,bonide
28,upc,1.0,<NA>,11.0,7489057,9821599,5540461,WM700018159,WM700018159,737025181598,...,737025181598,737025181598,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
30,upc,1.0,<NA>,11.0,7489073,9821600,5540462,WM700018172,WM700018172,737025181727,...,737025181727,737025181727,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
32,upc,1.0,<NA>,11.0,7489081,9821601,5541322,WM700051171,WM700051171,737025511548,...,737025511548,737025511548,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
34,upc,1.0,<NA>,11.0,7489099,9821602,5540464,WM700051184,WM700051184,737025511845,...,737025511845,737025511845,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
36,upc,1.0,<NA>,11.0,7489107,9821603,5529022,WM700051223,WM700051223,737025018764,...,737025018764,737025018764,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
38,upc,1.0,<NA>,11.0,7489115,9821604,5530118,WM700051224,WM700051224,737025018771,...,737025018771,737025018771,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi
40,upc,1.0,<NA>,11.0,8850729,9821701,5539598,WT760051161,WT760051161,737025511616,...,737025511616,737025511616,bona,bona,bonakemi,bona kemi,bona,bona,bona,bona kemi


In [16]:
multi_matching

,key,trust,master,type,item_code_v,prod_id_v,prod_id_m,mpn_v,mpn_m,upc_v,...,upc_v_mod,upc_m_mod,manuf_v_cl,raw_manuf_v_cl,manuf_m_cl,raw_manuf_m_cl,brand_v_cl,raw_brand_v_cl,brand_m_cl,raw_brand_m_cl


## Publish new products

In [17]:
print(pd.Timestamp.now())

metka_date = datetime.datetime.today().strftime("%Y%m%d")
suffix = 'new_products'

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params_test3}")
table = f"temp_dkh_vendor_{vendor_id}_{metka_date}_{suffix}"
new_products[['prod_id_v']].to_sql(table, con=engine, schema='dbo', if_exists='replace', index=False, chunksize=200, method='multi')
engine.dispose()
print(pd.Timestamp.now())

2023-10-25 12:23:05.029327
2023-10-25 12:23:08.941021


In [18]:
%%time
server = 
database = 
username = 
password = 
cnxn = pyodbc.connect(f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}")
cursor = cnxn.cursor()
query1 = f"""
            Delete pmc
            FROM products p
            JOIN products_matching_conflicts pmc ON pmc.product_id = p.id
            JOIN test3.dbo.{table} t ON t.prod_id_v = p.id
        """

cursor.execute(query1)

cnxn.commit()
cursor.close()
cnxn.close()


CPU times: total: 0 ns
Wall time: 1.3 s


In [19]:
print('NEED in VM push FORCE BUTTON')

NEED in VM push FORCE BUTTON


In [20]:
%%time

cnxn = pyodbc.connect(f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}")
cursor = cnxn.cursor()

query2 = f"""
            UPDATE pp SET pp.processing_stage_id = 1
            FROM products p
            JOIN processing_products pp ON pp.product_id = p.id AND pp.processing_stage_id = 4
            JOIN test3.dbo.{table} t ON t.prod_id_v = p.id
        """


query3 = f"""
            drop table test3.dbo.{table}
        """
cursor.execute(query2)
cursor.execute(query3)

cnxn.commit()
cursor.close()
cnxn.close()

CPU times: total: 0 ns
Wall time: 2.46 s


## Publish single matching products

In [21]:
%%time
print(pd.Timestamp.now())

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params_test3}")


metka_date = datetime.datetime.today().strftime("%Y%m%d")
suffix = 'single_matching'

# table = f"temp_dkh_vendor_{vendor_id}_{metka_date}_{suffix}"
single_matching[['prod_id_v', 'prod_id_m']].to_sql(table, con=engine, schema='dbo', if_exists='replace', index=False, chunksize=200, method='multi')
engine.dispose()
print(pd.Timestamp.now())

2023-10-25 12:37:22.955585
2023-10-25 12:37:26.241912
CPU times: total: 0 ns
Wall time: 3.29 s


In [22]:
%%time

cnxn = pyodbc.connect(f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}")
cursor = cnxn.cursor()
query1 = f"""
            Delete pmc
            FROM products p
            JOIN products_matching_conflicts pmc ON pmc.product_id = p.id
            JOIN test3.dbo.{table} t ON t.prod_id_v = p.id
        """

query2 = f"""
            UPDATE p
                SET p.master_product_id = t.prod_id_m
            FROM products p
            JOIN test3.dbo.{table} t ON t.prod_id_v = p.id
        """



cursor.execute(query1)
cursor.execute(query2)

cnxn.commit()
cursor.close()
cnxn.close()

CPU times: total: 0 ns
Wall time: 1.08 s


In [23]:
print('NEED in VM push FORCE BUTTON')

NEED in VM push FORCE BUTTON


In [24]:
%%time

cnxn = pyodbc.connect(f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}")
cursor = cnxn.cursor()

query2 = f"""
            UPDATE pp SET pp.processing_stage_id = 1
            FROM products p
            JOIN processing_products pp ON pp.product_id = p.id AND pp.processing_stage_id = 4
            JOIN test3.dbo.{table} t ON t.prod_id_v = p.id
        """


query3 = f"""
            drop table test3.dbo.{table}
        """
cursor.execute(query2)
cursor.execute(query3)

cnxn.commit()
cursor.close()
cnxn.close()

CPU times: total: 0 ns
Wall time: 694 ms


## Publish single products Equivalents

In [17]:
%%time
print(pd.Timestamp.now())

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params_test3}")


metka_date = datetime.datetime.today().strftime("%Y%m%d")
suffix = 'product_equivalents'

table = f"temp_dkh_vendor_{vendor_id}_{metka_date}_{suffix}"
single_equivalents[['prod_id_v', 'prod_id_m']].to_sql(table, con=engine, schema='dbo', if_exists='replace', index=False, chunksize=200, method='multi')
engine.dispose()
print(pd.Timestamp.now())

2023-10-27 09:48:12.576004
2023-10-27 09:48:16.515100
CPU times: total: 46.9 ms
Wall time: 3.94 s


In [43]:
single_equivalents[['prod_id_v', 'prod_id_m']]

,prod_id_v,prod_id_m
129,9811036,5512887
139,9811069,5593783
136,9811070,5593910
174,9812433,5960123


In [18]:
%%time

cnxn = pyodbc.connect(f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}")
cursor = cnxn.cursor()
query1 = f"""
            Delete pmc
            FROM products p
            JOIN products_matching_conflicts pmc ON pmc.product_id = p.id
            JOIN test3.dbo.{table} t ON t.prod_id_v = p.id
        """

cursor.execute(query1)

cnxn.commit()
cursor.close()
cnxn.close()

CPU times: total: 15.6 ms
Wall time: 1.5 s


In [19]:
print('NEED in VM push FORCE BUTTON')

NEED in VM push FORCE BUTTON


In [20]:
%%time

cnxn = pyodbc.connect(f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}")
cursor = cnxn.cursor()

query1 = f"""
            UPDATE pp SET pp.processing_stage_id = 1
            FROM products p
            JOIN processing_products pp ON pp.product_id = p.id AND pp.processing_stage_id = 4
            JOIN test3.dbo.{table} t ON t.prod_id_v = p.id
        """


cursor.execute(query1)

cnxn.commit()
cursor.close()
cnxn.close()

CPU times: total: 0 ns
Wall time: 627 ms


In [21]:
print('NEED in VM publish products and wait end of processing')

NEED in VM publish products and wait end of processing


In [22]:
%%time
print(pd.Timestamp.now())

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params_VM}")

query1 = f"""
            SELECT p.master_product_id, t.prod_id_m
            FROM products p
            JOIN test3.dbo.{table} t ON t.prod_id_v = p.id
            WHERE p.vendor_id = {vendor_id} and p.master_product_id IS NOT NULL
        """

product_equivalents = pd.read_sql(query1, con=engine)
engine.dispose()
product_equivalents.info(memory_usage='deep')
print(pd.Timestamp.now())


2023-10-27 09:49:41.780097
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   master_product_id  9 non-null      int64
 1   prod_id_m          9 non-null      int64
dtypes: int64(2)
memory usage: 272.0 bytes
2023-10-27 09:49:43.790452
CPU times: total: 0 ns
Wall time: 2.01 s


In [24]:
print(pd.Timestamp.now())

cnxn = pyodbc.connect(f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}")
cursor = cnxn.cursor()
## insert data
query1 = f"INSERT INTO DKHardware_NEW.dbo.equivalent_products VALUES (?, ?)"

query2 = f"INSERT INTO DKHardware_NEW.dbo.products_notes VALUES ( ?, GETUTCDATE(), 145, 'add equivalent_products with ' + CAST(? AS char))"


for row in tqdm(product_equivalents.loc[:, ('master_product_id', 'prod_id_m')].itertuples(index=False)):
    cursor.execute(query1, int(row.master_product_id), int(row.prod_id_m))
    cursor.execute(query2, int(row.master_product_id), int(row.prod_id_m))
    cursor.execute(query1, int(row.prod_id_m), int(row.master_product_id))
    cursor.execute(query2, int(row.prod_id_m), int(row.master_product_id))


cnxn.commit()
cursor.close()
cnxn.close()
print(pd.Timestamp.now())

2023-10-27 09:50:27.420933


9it [00:05,  1.69it/s]


2023-10-27 09:50:34.205347


## Select master products from same

In [32]:
%%time
print(pd.Timestamp.now())

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params_test3}")


metka_date = datetime.datetime.today().strftime("%Y%m%d")
suffix = 'multi_matching'

table = f"temp_dkh_vendor_{vendor_id}_{metka_date}_{suffix}"
multi_matching[['prod_id_v', 'prod_id_m']].to_sql(table, con=engine, schema='dbo', if_exists='replace', index=False, chunksize=200, method='multi')
engine.dispose()
print(pd.Timestamp.now())

2023-10-25 12:55:31.534793
2023-10-25 12:55:34.354696
CPU times: total: 15.6 ms
Wall time: 2.82 s


In [30]:
%%time

cnxn = pyodbc.connect(f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}")
cursor = cnxn.cursor()
query1 = f"""
            Delete pmc
            FROM products p
            JOIN products_matching_conflicts pmc ON pmc.product_id = p.id
            JOIN test3.dbo.{table} t ON t.prod_id_v = p.id
        """

query2 = f"""
            UPDATE p
                SET p.master_product_id = t.prod_id_m
            FROM products p
            JOIN test3.dbo.{table} t ON t.prod_id_v = p.id
        """


query3 = f"""
            drop table test3.dbo.{table}
        """
cursor.execute(query1)
cursor.execute(query2)
cursor.execute(query3)

cnxn.commit()
cursor.close()
cnxn.close()

CPU times: total: 0 ns
Wall time: 1.7 s


In [33]:
print(pd.Timestamp.now())

table = '"' + table + '"'
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params_DK}")

query_uom = f"""
            SELECT t.prod_id_v, t.prod_id_m, p.product_hold, cnt_ven, cnt_mrkt, cnt_ordr, cnt_wh, sum_stck, p.inbound_stock AS inb_stck
            FROM test3.dbo.{table} t
            --JOIN VendorMigrator.dbo.products vp on vp.id = t.prod_id_v
            JOIN DKHardware_NEW.dbo.products p ON p.product_ID = t.prod_id_m
            LEFT JOIN
                (
                SELECT t.prod_id_m, count(pv.id) AS cnt_ven
                FROM test3.dbo.{table} t
                JOIN DKHardware_NEW.dbo.products_vendors pv ON pv.product_id = t.prod_id_m
                GROUP BY t.prod_id_m
                ) c_ven ON c_ven.prod_id_m = t.prod_id_m
            LEFT JOIN
                (
                SELECT t.prod_id_m, count(pm.marketplace_id) AS cnt_mrkt
                FROM test3.dbo.{table} t
                JOIN DKHardware_NEW.dbo.product_marketplaces pm ON pm.product_id = t.prod_id_m
                GROUP BY t.prod_id_m
                ) c_mr ON c_mr.prod_id_m = t.prod_id_m
            LEFT JOIN
                (
                SELECT t.prod_id_m, count(sc.cart_ID) AS cnt_ordr
                FROM test3.dbo.{table} t
                JOIN DKHardware_NEW.dbo.shopping_cart sc ON sc.product_ID = t.prod_id_m
                GROUP BY  t.prod_id_m
                ) c_ord ON c_ord.prod_id_m = t.prod_id_m
            LEFT JOIN
                (
                SELECT t.prod_id_m, count(ps.location_id) AS cnt_wh, sum(ps.stock) AS sum_stck
                FROM test3.dbo.{table} t
                JOIN DKHardware_NEW.dbo.products_stock ps ON ps.product_id = t.prod_id_m AND ps.stock != 0
                GROUP BY  t.prod_id_m
                ) c_st ON c_st.prod_id_m = t.prod_id_m
            """

prod_marker_order = pd.read_sql(query_uom, con=engine)
engine.dispose()
prod_marker_order.loc[(prod_marker_order['inb_stck']==0), 'inb_stck'] = np.NaN
prod_marker_order.info(memory_usage='deep')
print(pd.Timestamp.now())

2023-10-25 12:55:38.398621
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   prod_id_v     9 non-null      int64  
 1   prod_id_m     9 non-null      int64  
 2   product_hold  9 non-null      bool   
 3   cnt_ven       9 non-null      int64  
 4   cnt_mrkt      0 non-null      object 
 5   cnt_ordr      1 non-null      float64
 6   cnt_wh        0 non-null      object 
 7   sum_stck      0 non-null      object 
 8   inb_stck      0 non-null      float64
dtypes: bool(1), float64(2), int64(3), object(3)
memory usage: 1.1 KB
2023-10-25 12:55:40.866672


In [34]:

single_master_mtch = pd.DataFrame()
debug_master = pd.DataFrame()

for VM_id in prod_marker_order['prod_id_v'].unique():

    flt_vm_id = prod_marker_order[prod_marker_order['prod_id_v'] == VM_id].copy()

    #filter with market
    local_flt = flt_vm_id[flt_vm_id['cnt_mrkt'].notna()].copy()
    if local_flt.shape[0] == 1:
        single_master_mtch = pd.concat([single_master_mtch, local_flt])
        continue
    elif local_flt.shape[0] > 1:
        local_flt.loc[:,'debug_lvl'] = 'market'
        debug_master = pd.concat([debug_master, local_flt.set_index(['debug_lvl', local_flt.index])])
        continue

    #filter with order
    local_flt = flt_vm_id[flt_vm_id['cnt_ordr'].notna()].copy()
    if local_flt.shape[0] == 1:
        single_master_mtch = pd.concat([single_master_mtch, local_flt])
        continue
    elif local_flt.shape[0] > 1:
        local_flt.loc[:,'debug_lvl'] = 'order'
        debug_master = pd.concat([debug_master, local_flt.set_index(['debug_lvl', local_flt.index])])
        continue

    #filter from not hold with max count vendor
    local_flt = flt_vm_id.loc[flt_vm_id['product_hold']==False].copy()
    if local_flt.empty:
        flt_vm_id.loc[:,'debug_lvl'] = 'hold'
        debug_master = pd.concat([debug_master, flt_vm_id.set_index(['debug_lvl', flt_vm_id.index])])
    else:
        local_flt = local_flt.loc[local_flt['cnt_ven'] == local_flt['cnt_ven'].max()].copy()
        if local_flt.shape[0] == 1:
            single_master_mtch = pd.concat([single_master_mtch, local_flt])
            continue
        elif local_flt.shape[0] > 1:
            local_flt.loc[:, 'all_stck'] = flt_vm_id['sum_stck'] + flt_vm_id['inb_stck']
            local_flt = local_flt.loc[local_flt['all_stck'] == local_flt['all_stck'].max()].copy()
            if local_flt.empty:
                continue
            elif local_flt.shape[0] == 1:
                single_master_mtch = pd.concat([single_master_mtch, local_flt])
                continue
            elif local_flt.shape[0] > 1:
                local_flt.loc[:,'debug_lvl'] = 'prodven_stock'
                debug_master = pd.concat([debug_master, local_flt.set_index(['debug_lvl', local_flt.index])])
                continue

    flt_vm_id.loc[:,'debug_lvl'] = 'other'
    debug_master = pd.concat([debug_master, flt_vm_id.set_index(['debug_lvl', flt_vm_id.index])])


In [36]:
# debug_master#.loc[debug_master['prod_id_v']==9800230]
single_master_mtch

,prod_id_v,prod_id_m,product_hold,cnt_ven,cnt_mrkt,cnt_ordr,cnt_wh,sum_stck,inb_stck
6,9821575,5515409,False,1,None,1.0,None,None,NaN


In [322]:
cand_confl_re_flt.loc[cand_confl_re_flt['prod_id_v']==9801066]

,key,trust,master,type,item_code_v,prod_id_v,prod_id_m,mpn_v,mpn_m,upc_v,upc_m,manuf_v,raw_manuf_v,brand_v,raw_brand_v,manuf_m,brand_m,raw_manuf_m,raw_brand_m,ratio_rp,ratio_dkp,ratio_cp,price_v,ret_price_m,price_m,title_v,title_m,cost_v,cost_m,MinOrV,MinOrM,uom_v,uom_m,manuf_id_v,manuf_id_m,raw_manuf_id_v,brand_id_v,brand_id_m,raw_brand_id_v,mpn_v_mod,item_m,ven_id,cnt_abc,upc_v_mod,upc_m_mod,manuf_v_cl,raw_manuf_v_cl,manuf_m_cl,raw_manuf_m_cl,brand_v_cl,raw_brand_v_cl,brand_m_cl,raw_brand_m_cl
460,upc,1,<NA>,11.0,4002648,9801066,5597755,B 705,B 705,049793007056,049793007056,Prime-Line Products,PRIME LINE PRODUCTS,Prime-Line,Prime-Line,PRIME-LINE PRODUCTS/SLIDE-CO,Prime-Line,PRIME-LINE PRODUCTS/SLIDE-CO,Prime-Line,0.184184,0.255027,0.266944,8.15,9.99,10.94,"Bug Seal, 3/4 to 1-3/8 in W, 7 in L, Vinyl",Bug Seal Replacement Matte Black Vinyl Matte,4.2224,5.76,1,1.0,Card,Each,25242,31021,46901,2324,2324,18841,NaN,5492111,116,NaN,49793007056,49793007056,prime line,prime line,prime line slide,prime line slide,prime line,prime line,prime line,prime line
461,upc,1,<NA>,11.0,4002648,9801066,5596055,B 705,B705,049793007056,049793007056,Prime-Line Products,PRIME LINE PRODUCTS,Prime-Line,Prime-Line,PRIME-LINE PRODUCTS/SLIDE-CO,Prime-Line,PRIME-LINE PRODUCTS/SLIDE-CO,Prime-Line,0.184184,0.39941,0.040364,8.15,9.99,13.57,"Bug Seal, 3/4 to 1-3/8 in W, 7 in L, Vinyl",Door Sweep Black Vinyl Black,4.2224,4.4,1,1.0,Card,Each,25242,31021,46901,2324,2324,18841,NaN,5034310,116,NaN,49793007056,49793007056,prime line,prime line,prime line slide,prime line slide,prime line,prime line,prime line,prime line


In [321]:
cand_confl_re_flt.iloc[21]

key                                                             upc
trust                                                             1
master                                                         <NA>
type                                                           11.0
item_code_v                                                 7484033
prod_id_v                                                   9804123
prod_id_m                                                   3656358
mpn_v                                                    48-73-5032
mpn_m                                                    48-73-5032
upc_v                                                  045242552580
upc_m                                                  045242552580
manuf_v                                                   Milwaukee
raw_manuf_v                                               MILWAUKEE
brand_v                                                   Milwaukee
raw_brand_v                                     

In [495]:
single_master_mtch.loc[:,'debug_lvl'] = 'sd'
single_master_mtch.set_index(['debug_lvl', single_master_mtch.index])
# single_master_mtch

prod_id_v  prod_id_m  product_hold  cnt_ven  cnt_mrkt  cnt_ordr  \
level                                                                         
sd    3923    8726841    5511765         False        2       8.0       NaN   
      5788    8648978    5716749         False        2       NaN       NaN   

            cnt_wh  sum_stck  inb_stck  
level                                   
sd    3923     NaN       NaN       NaN  
      5788     NaN       NaN       NaN

In [505]:
t = prod_marker_order[prod_marker_order['prod_id_v'] == 8726841]
display(t)
t.loc[t['cnt_ven'] == t.loc[t['product_hold']==False, 'cnt_ven'].max()]#.to_frame().T

,prod_id_v,prod_id_m,product_hold,cnt_ven,cnt_mrkt,cnt_ordr,cnt_wh,sum_stck,inb_stck
1,8726841,256537,True,3,NaN,NaN,NaN,NaN,NaN
3923,8726841,5511765,False,2,8.0,NaN,NaN,NaN,NaN


,prod_id_v,prod_id_m,product_hold,cnt_ven,cnt_mrkt,cnt_ordr,cnt_wh,sum_stck,inb_stck
3923,8726841,5511765,False,2,8.0,NaN,NaN,NaN,NaN


### SKLAD

In [978]:
big_brand = cand_confl_re[(cand_confl_re['brand_v'].isna())].value_counts('raw_brand_v')
display(big_brand)

raw_brand_v
Santas Forest       777
MIDWEST FASTENER    669
ProSource           504
Richard             406
Vulcan              396
                   ... 
FREE ALL              1
FibaFuse              1
SERGEANT'S PET        1
GOJO INDUSTRIES       1
SikaQuick             1
Length: 590, dtype: int64

In [979]:
res = pd.DataFrame()
for brand in big_brand.index:
    # print(manuf)
    # pat = cand_confl_re.loc[cand_confl_re['raw_brand_v']==brand, 'raw_brand_v_pat'].iloc[0]
    # print(pat)
    temp_df = cand_confl_re[(cand_confl_re['raw_brand_v']==brand) & (cand_confl_re['brand_m'].str.contains(brand, case=False, regex=False))].drop_duplicates(subset=['brand_m','raw_brand_v']).sort_values('brand_m').iloc[:6]
    if not temp_df.empty:
        res = pd.concat([res, temp_df])
display(res)

,key,trust,master,type,prod_id_v,prod_id_m,mpn_v,mpn_m,upc_v,upc_m,manuf_v,manuf_m,raw_manuf_v,brand_v,brand_m,raw_brand_v,ratio_r,ratio_m,price_v,ret_price_m,price_m,title_v,title_m,cost_v,cost_m,MinOrV,MinOrM,unit_v,manuf_id_v,manuf_id_m,raw_manuf_id_v,brand_id_v,brand_id_m,raw_brand_id_v,mpn_mod,item_code_v,raw_manuf_v_cl,raw_manuf_v_pat,manuf_m_cl
2360,mpn,1,<NA>,2,8745606,5574446,RL-100 2,RL-100 2,738312012045,<NA>,<NA>,THE MAURICE FRANKLIN LOUVER CO,MAURICE FRANKLIN,<NA>,Maurice Franklin Louver,Maurice Franklin,0.2595,0.065801,10.79,7.99,11.55,"Screen Louver, 2.391 W, Aluminum, Mill - pack ...","Mini Louver 2"" W X 2"" L Bright Silver Aluminum...",5.819,6.08,1,1,BG,<NA>,31235,47388,<NA>,10007,18649,rl1002,8634024,maurice franklin,(maurice).+(franklin),maurice franklin louver
8936,mpn,1,<NA>,2,8701029,5525306,CHH-000A-BL-070,CHH-000A-BL-070,664911300149,<NA>,ROCKY BRANDS US LLC,HONEYWELL SAFETY PRODUCTS USA,ROCKY BRANDS INC,<NA>,The Original Muck Boot Company,Muck,0.220979,0.005914,141.19,109.99,142.03,"CHORE Series Boots, 7, Black, Rubber Upper",Boots Chore Hi Men's 7 US Black Black,91.6115,79.79,1,1,PR,31913,31096,46708,<NA>,8901,19412,chh000abl070,4997086,rocky brands,(rocky).+(brands),honeywell safety
6779,mpn,1,<NA>,11,8755413,5724281,754049-0806,754049-0806,719852939061,719852939061,Anderson Metals,Anderson Metals,ANDERSON METAL,<NA>,Anderson Metals,ANDERSON METAL,0.105131,0.062937,7.15,7.99,6.7,ELBOW 1/2FLARE X 3/8MPT LF,"Pipe Fittings, Flare Elbow, Lead-Free Brass, 1...",3.328,3.4362,5,1,EA,516,516,47729,<NA>,1933,19084,7540490806,8979577,anderson metal,(anderson).+(metal),anderson metals
23996,mpn,1,<NA>,11,8674079,5951175,100532366,100532366,021496277554,21496277554,PENNINGTON SEED INC,PENNINGTON SEED INC,PENNINGTON SEED,<NA>,PENNINGTON SEED VIRGINIA DIV,PENNINGTON SEED,0.001153,0.085559,52.05,51.99,56.92,SEED ZOYSIA ZENITH W/MULCH 5LB,"Zenith Zoysia Grass Seed and Mulch, 5-Lbs.",31.3231,31.05,1,1,BG,31004,31004,46629,<NA>,6224,17624,100532366,0276956,pennington seed,(pennington).+(seed),pennington seed
7750,mpn,1,<NA>,11,8678305,6536161,GA5578IV,GA5578IV,093598255785,93598255785,RALPH FRIEDLAND & BROTHERS,RALPH FRIEDLAND & BROTHERS,RALPH FRIEDLAND & BROS,<NA>,RALPH FRIEDLAND & BROTHERS,ralph friedland,0.170973,0.025199,31.35,25.99,30.56,"Galaxy Roller Shade, 78 in L, 55 in W, Vinyl, ...","Galaxy Roller Shade, Light Filtering, White, 5...",16.8081,16.0839,1,1,EA,26989,26989,47815,<NA>,4308,19271,ga5578iv,5277736,ralph friedland bros,(ralph).+(friedland).+(bros),ralph friedland brothers
25300,mpn,1,<NA>,2,8708129,5962620,203-060-005,203060005,062338009728,62338009728,ROYAL OAK ENTERPRISES LLC,ROYAL OAK SALES,ROYAL OAK ENTERPRISES,<NA>,ROYAL OAK SALES,ROYAL OAK,0.328007,0.046512,11.89,7.99,12.47,24/1 Paraffin Wax,"Household Paraffin Wax, 1-Lb.",5.7757,6.5619,1,1,BX,30335,32359,47599,<NA>,12229,17828,203060005,7028731,royal oak enterprises,(royal).+(oak).+(enterprises),royal oak sales
4452,mpn,1,<NA>,2,8747937,6536398,RK-3/4IC100,RK-3/4IC100,694585031001,694585031001,<NA>,ELECTRIC EEL MFG CO,ELECTRIC EEL,<NA>,ELECTRIC EEL MFG CO,ELECTRIC EEL,0.135156,0.16144,2624.75,2270.0,2201.01,"Drain Cleaner, Electric, 3/4 in Dia Cable, 100...","3/4""x100' Drain Cleaner",1756.5184,1467.34,1,1,EA,<NA>,32190,47095,<NA>,12189,18272,rk34ic100,9306945,electric eel,(electric).+(eel),electric eel mfg
25141,mpn,1,<NA>,11,8680795,4116108,AZB70422,AZB70422,028078704224,<NA>,GRO-WELL BRANDS INC,GRO-WELL BRANDS INC,GRO-WELL BRANDS,<NA>,GRO-WELL BRANDS INC,GRO-WELL,0.450974,0.59176,16.35,29.78,40.05,"All-Purpose Fertilizer, 20 lb Bag, Solid, 10-1...",Azbest 10-10-10 20# All Purpose Fert,9.4536,21.65,1,1,BG,27850,27850,47499,<NA>,5632,17705,azb70422,5157888,gro well brands,(gro).+(well).+(brands),gro well brands
6794,mpn,1,<NA>,11,8655500,6546205,INS541LD,INS541LD,729477005409,729477005409,U.S. GREENFIBER LLC,U.S. GREENFIBER LLC,U.S. GREENFIBER,<NA>,U.S. GREENFIBER LLC,Greenfiber,0.266667,0.111896,16.35,11.99,18.41,"Blow-In Insulation, R13 t

In [86]:
# pd.Series(tt).sort_values()
bad_type = cand_confl_re[((cand_confl_re['type'].isna()) | (cand_confl_re['type'] == 20))]
big_manuf = bad_type.value_counts('manuf_v')

display(big_manuf)

# manufs = bad_type.loc[:, ['manuf_v', 'manuf_v_cl', 'manuf_id_v', 'manuf_m', 'manuf_id_m', 'manuf_m_cl']].drop_duplicates().copy()
#
# res = pd.DataFrame()
# for row in manufs.loc[:, ['manuf_v', 'manuf_id_v']].drop_duplicates().itertuples():
#     temp_df = bad_type[
#         (bad_type['manuf_v'] == row.manuf_v) & (bad_type['manuf_m'].str.contains(row.manuf_v, case=False, regex=False))]
#     if not temp_df.empty:
#         res = pd.concat([res, temp_df])

##########
tt = bad_type.loc[(bad_type['manuf_v']=='BARON MFG CO LLC'), 'brand_m'].unique().copy()# & (bad_type['manuf_m'].str.contains(row.manuf_v, case=False, regex=False))]
display(pd.Series(tt).sort_values())


###########

bad_type = cand_confl_re[((cand_confl_re['type'].isna()) | (cand_confl_re['type']==20))]
big_manuf = bad_type.value_counts('manuf_v')

display(big_manuf)


manufs = bad_type.loc[:, ['manuf_v', 'manuf_v_cl', 'manuf_id_v', 'manuf_m', 'manuf_id_m', 'manuf_m_cl']].drop_duplicates().copy()

res = pd.DataFrame()
for row in manufs.loc[:, ['manuf_v', 'manuf_id_v']].drop_duplicates().itertuples():
    temp_df = bad_type[(bad_type['manuf_v']==row.manuf_v) & (bad_type['manuf_m'].str.contains(row.manuf_v, case=False, regex=False))]
    if not temp_df.empty:
        res = pd.concat([res, temp_df])

display(res)
#
res = pd.DataFrame()
for manuf in big_manuf.index:
    # print(manuf)
    # pat = cand_confl_re.loc[cand_confl_re['manuf_v']==manuf, 'manuf_v_pat'].iloc[0]
    # print(pat)
    temp_df = bad_type[(bad_type['manuf_v']==manuf) & (bad_type['manuf_m'].str.contains(manuf, case=False, regex=False))].drop_duplicates(subset=['manuf_m','manuf_v']).sort_values('manuf_m').iloc[:6]
    if not temp_df.empty:
        res = pd.concat([res, temp_df])
display(res)
display(cand_confl_re[(cand_confl_re['type']!=111) & (cand_confl_re['raw_manuf_v']==manuf)].sort_values('brand_m').iloc[:40].drop_duplicates(subset=['manuf_m','raw_manuf_v']).sort_values('manuf_m').iloc[:6])

manuf_v
MULTINAUTIC INTL                   141
Norton                              68
FLP                                 64
JONATHAN GREEN TURF                 58
WHIZZ                               54
                                  ... 
CONTINENTAL COMMERCIAL PRODUCTS      1
Motsenbocker's Liftoff               1
Milwaukee                            1
REESE TOWPOWER                       1
PROTECTO WRAP COMPANY                1
Length: 223, dtype: int64

In [1078]:
good_type = [1,11,12,77,777]
# cand_confl_re_flt = cand_confl_re[cand_confl_re['type'].isin(good_type)].copy()
cand_confl_re_flt = cand_confl_re[cand_confl_re['type']!=20].copy()
calc_vers_confl = pd.DataFrame(cand_confl_re_flt.loc[:, 'prod_id_v'].value_counts()).reset_index()
# filter
flt_vers_confl = calc_vers_confl.loc[calc_vers_confl['prod_id_v'] == 1][['index']]
# display
all_flt = cand_confl_re_flt[cand_confl_re_flt['prod_id_v'].isin(flt_vers_confl['index'])].sort_values('prod_id_v')
# all_flt.loc[all_flt['master']==True].to_csv('remap_to_master_doorcp.csv')
# remap = all_flt.loc[(all_flt['master']==True) | (all_flt['master']==False), 'prod_id_v'].unique()
# temp_1 = all_flt[~all_flt['prod_id_v'].isin(remap)]#.head(10)
single_prod_v = all_flt.copy()

print(single_prod_v.shape)
single_prod_v.iloc[0:30]

(44216, 39)


,key,trust,master,type,prod_id_v,prod_id_m,mpn_v,mpn_m,upc_v,upc_m,manuf_v,manuf_m,raw_manuf_v,brand_v,brand_m,raw_brand_v,ratio_r,ratio_m,price_v,ret_price_m,price_m,title_v,title_m,cost_v,cost_m,MinOrV,MinOrM,unit_v,manuf_id_v,manuf_id_m,raw_manuf_id_v,brand_id_v,brand_id_m,raw_brand_id_v,mpn_mod,item_code_v,raw_manuf_v_cl,raw_manuf_v_pat,manuf_m_cl
98171,mpn,1,<NA>,11,8647508,5606769,23577,23577,036577001270,<NA>,<NA>,BLOUNT INC,OREGON DISTRIBUTION,Oregon,Oregon,Oregon,0.171429,0.547465,10.85,8.99,4.91,"Chainsaw File, Round, Steel - pack of 2",Chainsaw Chain File Sure Sharp,3.5044,2.52,1,1,CRD,<NA>,31384,47064,7012,7012,17701,23577,0528737,oregon distribution,(oregon).+(distribution),blount
42448,mpn,1,<NA>,11,8647512,5031951,CWKWS-58-010,CWKWS-58-010,781513651070,781513651070,Mechanix Wear,Mechanix Wear,MECHANIX WEAR,Mechanix Wear,Mechanix Wear,MECHANIX WEAR,<NA>,0.142889,38.39,<NA>,44.79,"ColdWork WindShell Winter Gloves, Men's, L, 13...",Wind Shell Grey/Black Large Cold Weather Gloves,23.103,24.21,1,1,PR,55,55,47094,1694,1694,18270,cwkws58010,0743435,mechanix wear,(mechanix).+(wear),mechanix wear
34732,mpn,1,<NA>,11,8647513,5590295,8125,8125,050169001257,<NA>,HUBBELL ELECTRICAL PRODUCTS,HUBBELL ELECTRICAL PRODUCTS,HUBBELL ELECTRICAL PRODUCTS,RACO,RACO,RACO,0.056225,0.169611,2.35,2.49,2.83,"Octagonal Box, 3.563 in OAW, 1.813 in OAD, 3.5...",Junction Box 15-1/2 cu in Octagon Steel 1 gang...,1.5142,1.449,1,1,EA,30859,30859,47188,2327,2327,18090,8125,1781129,hubbell electrical,(hubbell).+(electrical),hubbell electrical
44396,mpn,1,<NA>,11,8647514,3148708,48-00-1301,48-00-1301,045242081745,045242081745,Milwaukee,Milwaukee Electric Tool Corporation,MILWAUKEE,Milwaukee,Milwaukee,Milwaukee,<NA>,0.021878,39.79,<NA>,40.68,"Reciprocating Saw Blade, 3/4 in W, 9 in L, 5 T...",9 in. 5 Teeth per in. Pruning SAWZALL Reciproc...,23.0256,21.99,1,1,CRD,5,25076,47417,1698,1698,18690,48001301,0508994,milwaukee,(milwaukee),milwaukee electric tool
55281,mpn,1,<NA>,11,8647518,5585915,UNA07,UNA-07,777808130467,<NA>,Boshart Industries,GARMEL HOLDING INC,BOSHART INDUSTRIES,Boshart Industries,Boshart Industries,Boshart,0.323741,0.191489,0.94,1.39,0.76,"UNA-07 Pipe Adapter, 1/2 in, Insert, MPT, Nylon","Adapter 3/4"" Insert in. X 3/4"" D FTG Nylon",0.3432,0.39,250,1,EA,565,31035,47152,1966,1966,18475,una07,0278283,boshart industries,(boshart).+(industries),garmel holding
85905,mpn,1,<NA>,11,8647521,5715026,BL-BMTN-W,BL-BMTN-W,070686009918,<NA>,Westek,AMERTAC HOLDINGS INC,WESTEK,Westek,Westek,Westek,0.123532,0.117003,20.15,22.99,22.82,"Motion Activated Light Bar, AA Battery, LED La...","Light Bar 12.0"" L White Battery Powered LED 15...",10.8917,12.01,1,1,CRD,2024,30987,47132,2662,2662,18308,blbmtnw,2757243,westek,(westek),amertac holdings
30235,mpn,1,<NA>,11,8647523,5557021,71178,71178,044600711782,<NA>,KINGSFORD CO,KINGSFORD CO,KINGSFORD,KINGSFORD,KINGSFORD,KINGSFORD,0.239594,0.15812,9.85,7.49,11.7,"Charcoal Lighter Fluid, Liquid, 64 oz",Charcoal Lighter Fluid 64 oz,6.5728,6.16,6,6,EA,31518,31518,47122,496,496,18299,71178,0500959,kingsford,(kingsford),kingsford
44232,mpn,1,<NA>,11,8647524,5597461,49-90-2034,49-90-2034,045242610396,<NA>,Milwaukee,Milwaukee Electric Tool Corporation,MILWAUKEE,Milwaukee,Milwaukee,Milwaukee,0.219285,0.114156,33.29,25.99,37.58,"AIR-TIP Series Non-Marring Utility Nozzle Kit,...",Vacuum Nozzle Air-Tip Shop Vac Non Marring Uti...,20.0312,19.78,1,1,ST,5,25076,47417,1698,1698,18690,49902034,2101889,milwaukee,(milwaukee),milwaukee electric tool
129618,mpn,1,<NA>,11,8647527,5527187,73125,73125,720547731259,<NA>,<NA>,Rust-Oleum Corporation,MEAN GREEN,Mean Green,Mean Green,Mean Green,0.083176,0.018557,4.85,5.29,4.76,"Cleaner, 48 oz, Liquid, Characteristic",Multi-Purpose Cleaner Pine Scent Liquid 48 oz,2.5958,2.44,1,8,EA,<NA>,25319,47596,10763,10763,18915,73125,2406239,mean green,(mean).+(green),rust oleum
77756,mpn,1,<NA>,11,8647530,5607698,N 6545,N 6545,049793065452,<NA>,Prime-Line Products,PRIME-LINE PRODUCTS/SLIDE-CO,PRIME LIN

In [ ]:
# # %%time
# ## old version one step regex replacment, now has done two step in def creat_mapping_conflicts
# prod_vendor_old = prod_vendor.copy()
# prod_DKH_old = prod_DKH.copy()

# prod_vendor_old['mpn_mod'] = prod_vendor_old['mpn_v'].apply(replace_pattern, args=('low',))
# prod_DKH_old['mpn_mod'] = prod_DKH_old['mpn_m'].apply(replace_pattern, args=('low',))

# result_old = find_conflicts(prod_vendor_old, prod_DKH_old, ratio=0.2)

In [ ]:

#### insert row=constant in df none increment index
# pd.concat([result, pd.DataFrame([[np.nan]*result.shape[1]], columns=result.columns)], ignore_index=True)
#### insert row=constant in df increment index
# rr = result.copy()#.iloc[result.iloc[-1]]
# rr.reset_index(drop=True, inplace=True)
# rr.loc[rr.iloc[-1].name + 1,:] = np.nan
# rr

In [ ]:

#3cell
# val = [np.nan, np.nan, True, 7429711, 5662559]
# val.extend([np.nan]*(result.shape[1]-5))

# rows_vendor_master = [[7429711, 5662559]]
# def add_rows(df, rows):
#     count = 0
#     for row in rows:
#         if row[0] in df.loc[df['master']==True, 'prod_id_v'].values:
#             continue
#         else:
#             val = [1, 1]
#             val.extend([True, row[0], row[1]])
#             val.extend([np.nan]*(df.shape[1]-5))
#             df = pd.concat([df, pd.DataFrame([val], columns=result.columns)], ignore_index=True)
#             count += 1
#     print(f'added {count} row(s)')
#     return df